In [ ]:
from gpiozero import PWMOutputDevice, DigitalOutputDevice
import math
import time

class Motor(object):
        MAX_SPEED = 1.0

        def __init__(self, pwm_pin, dir_pin):
                self.pwm_pin = PWMOutputDevice(pwm_pin)
                self.dir_pin = DigitalOutputDevice(dir_pin)
                self.speed = 0

        def setSpeed(self, speed):
                self.speed = speed
                dir_value = 1 if speed < 0 else 0
                speed = abs(speed)
                speed = min(speed, Motor.MAX_SPEED)
                self.pwm_pin.value = speed
                self.dir_pin.value = dir_value

class Motors(object):
        MAX_SPEED = 1.0

        def __init__(self):
                self.motor1 = Motor(12, 5)
                self.motor2 = Motor(13, 6)

        def setSpeeds(self, m1_speed, m2_speed):
                self.motor1.setSpeed(m1_speed)
                self.motor2.setSpeed(m2_speed)

class L3G4200D:

    def __init__(self, i2c_bus=1, addr=0x69):
        self.bus = SMBus(i2c_bus)
        self.addr = addr
        self.bus.write_byte_data(self.addr, 0x20, 0x0F)
        self.bus.write_byte_data(self.addr, 0x23, 0x30 | 0x80)
        self.lsb = 2000/32768 # COnfigured for 2000DPS scale
        time.sleep(1.0)
        self.x_offset = 0
        self.y_offset = 0
        self.z_offset = 0
        if os.path.exists('calib.yaml'):
            f = open('calib.yaml', 'r')
            calib_data = yaml.load(f)
            self.x_offset = calib_data['x_offset']
            self.y_offset = calib_data['y_offset']
            self.z_offset = calib_data['z_offset']
            f.close()
            self._calibrated = True
        else:
            self._calibrated = False

    def calibrated(self):
        return self._calibrated

    def calibrate(self, samples, delay_ms = 20):
        sample_captured = samples
        x_offset = 0
        y_offset = 0
        z_offset = 0
        while sample_captured > 0:
            x, y, z = self.read()
            sample_captured -= 1
            x_offset += x
            y_offset += y
            z_offset += z
            time.sleep(delay_ms/1000)
        #print(f'{selfx_offset} {self.y_offset} {self.z_offset}\n')
        self.x_offset = x_offset/samples
        self.y_offset = y_offset/samples
        self.z_offset = z_offset/samples
        data = {'x_offset' : self.x_offset, 'y_offset' : self.y_offset, 'z_offset' : self.z_offset}
        file_c = yaml.dump(data)
        with open('calib.yaml', 'w') as f:
            f.write(file_c)
            
    def read(self):
        #status = self.bus.read_byte_data(self.addr, 0x27)
        #if ((status >> 3) & 0x01) == 1 and ((status >> 7) & 0x01) == 1:
        reg_out = self.bus.read_i2c_block_data(self.addr, 0x28 | 0x80, 6)
        x, y, z = struct.unpack('<hhh', bytes(reg_out))
        return (x*self.lsb)-self.x_offset, (y*self.lsb)-self.y_offset, (z*self.lsb)-self.z_offset
        #return None                
                
class DifferentialRobot():
    
    def __init__(self, motors, L):
        self.motors = motors
        self.wheel_base = L
        self.linear_velocity = 0
        self.angular_velocity = 0
    
    def set_angular_velocity(self, angular_velocity):
        self.drive(self.linear_velocity, angular_velocity)
        
    def set_linear_velocity(self, linear_velocity):
        self.drive(linear_velocity, self.angular_velocity)
        
    def drive(self, linear_velocity, angular_velocity ):
        self.linear_velocity = linear_velocity
        self.angular_velocity = angular_velocity
        r_speed = linear_velocity + ((self.wheel_base/2) * angular_velocity)
        l_speed = linear_velocity - ((self.wheel_base/2) * angular_velocity)
        r_speed =  -r_speed #Robot camera sets the forward direction
        l_speed =  -l_speed
        r_speed = max(min(r_speed, 1.0), -1.0)
        l_speed = max(min(l_speed, 1.0), -1.0)
        self.motors.setSpeeds(r_speed, l_speed)
    
    def stop(self):
        self.motors.setSpeeds(0, 0)
                
robot_motors = Motors()
robot = DifferentialRobot(robot_motors, 0.09)

In [ ]:
from pyPS4Controller.controller import Controller


class MyController(Controller):

    def __init__(self, **kwargs):
        Controller.__init__(self, **kwargs)
    
    def on_L3_up(self, value):
        robot.set_linear_velocity(-value/(2**15))

    def on_L3_down(self, value):
        robot.set_linear_velocity(-value/(2**15))
    
    def on_L3_left(self, value):
        robot.set_angular_velocity(-value/(2**12))

    def on_L3_right(self, value):
        robot.set_angular_velocity(-value/(2**12))
    
    def on_L3_y_at_rest(self):
        robot.set_linear_velocity(0)
    
    def on_L3_x_at_rest(self):
        robot.set_angular_velocity(0)


controller = MyController(interface="/dev/input/js0", connecting_using_ds4drv=False)
controller.listen()